This tutorial demonstrates how to download all SEC 10-K filings of Russell 3000 companies that were filed in a given timeframe, for example from 2015 to 2022.

First, we download the CSV file containing all recent holdings of the Russell 3000 index from [ishares.com](https://www.ishares.com/us/products/239714/ishares-russell-3000-etf).

![img](https://i.imgur.com/qGX0F76l.png)

<!-- The URL of the CSV file is: https://www.ishares.com/us/products/239714/ishares-russell-3000-etf/1467271812596.ajax?fileType=csv&fileName=IWV_holdings&dataType=fund&asOfDate=20221230. -->



### Download CSV file with Russell 3000 constituents, convert to DataFrame

In [2]:
import requests

url = 'https://www.ishares.com/us/products/239714/ishares-russell-3000-etf/1467271812596.ajax?fileType=csv&fileName=IWV_holdings&dataType=fund&asOfDate=20221230'
response = requests.get(url)

with open('russell-3000.csv', 'wb') as f:
    f.write(response.content)

After opening the CSV file in a text editor, we see that the file is not formatted correctly and needs to be cleaned before conveting the CSV file into a pandas DataFrame. The first 10 lines of metadata show a different structure than the actual holding data, starting at line 11. The file also contains a text block at the end that we need to remove.

<!-- ![incorrect-csv](https://i.imgur.com/fasJMXj.png) -->
![incorrect-csv](https://i.imgur.com/IxNALi9.png)

<small>(Metadata, line 1 to 10)</small>

![incorrect-csv-2](https://i.imgur.com/rF5or5f.png)

<small>(Text block after line 2611)</small>

The metadata section at the beginning and the text block at the end are separated by a seemingly empty row (line 10 and line 2612). Don't get fooled here, because the empty row actually represents the Unicode character `\xa0`. `\xa0` is a non-breaking space character in Unicode encoding. It is also known as a "hard space" or "fixed space" to create a space between words that cannot be broken by line breaks or word wraps.

In [3]:
# cleaning CSV file
import csv

with open('russell-3000.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    rows = list(reader)

empty_row_indicies = [i for i in range(len(rows)) if (len(rows[i]) == 0 or '\xa0' in rows[i])]

print('Empty rows:', empty_row_indicies)

start = empty_row_indicies[0] + 1
end = empty_row_indicies[1]
cleaned_rows = rows[start:end]

with open('russell-3000-clean.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(cleaned_rows)

Empty rows: [8, 2621]


In [6]:
import pandas as pd

# load Russell 3000 holdings CSV into a dataframe
holdings = pd.read_csv('./russell-3000-clean.csv')
holdings.head(10)

,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date
0,AAPL,APPLE INC,Information Technology,Equity,"559,365,151.11",5.16,"559,365,151.11","4,305,127.00",129.93,United States,NASDAQ,USD,1.0,USD,-
1,MSFT,MICROSOFT CORP,Information Technology,Equity,"513,917,712.42",4.74,"513,917,712.42","2,142,931.00",239.82,United States,NASDAQ,USD,1.0,USD,-
2,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,"213,823,596.00",1.97,"213,823,596.00","2,545,519.00",84.00,United States,NASDAQ,USD,1.0,USD,-
3,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Financials,Equity,"159,603,687.60",1.47,"159,603,687.60","516,684.00",308.90,United States,New York Stock Exchange Inc.,USD,1.0,USD,-
4,GOOGL,ALPHABET INC CLASS A,Communication,Equity,"151,996,026.75",1.40,"151,996,026.75","1,722,725.00",88.23,United States,NASDAQ,USD,1.0,USD,-
5,UNH,UNITEDHEALTH GROUP INC,Health Care,Equity,"142,028,859.84",1.31,"142,028,859.84","267,888.00",530.18,United States,New York Stock Exchange Inc.,USD,1.0,USD,-
6,GOOG,ALPHABET INC CLASS C,Communication,Equity,"135,557,878.61",1.25,"135,557,878.61","1,527,757.00",88.73,United States,NASDAQ,USD,1.0,USD,-
7,JNJ,JOHNSON & JOHNSON,Health Care,Equity,"133,147,817.70",1.23,"133,147,817.70","753,738.00",176.65,United States,New York Stock Exchange Inc.,USD,1.0,USD,-
8,XOM,EXXON MOBIL CORP,Energy,Equity,"130,179,148.40",1.20,"130,179,148.40","1,180,228.00",110.30,United States,New York Stock Exchange Inc.,USD,1.0,USD,-
9,JPM,JPMORGAN CHASE & CO,Financials,Equity,"112,022,178.30",1.03,"112,022,178.30","835,363.00",134.10,United States,New York Stock Exchange Inc.,USD,1.0,USD,-


### Download 10-K filings of Russell 3000 constituents

In the next step we download all 10-K filings for each ticker listed in the `holdings` DataFrame.

In [7]:
!pip install -q sec-api

The process looks like this:
- For each ticker, we download all 10-K URLs in a specified time frame, for example from 2015 to 2022.
- For each URL, we download the filing and save it to our local disk while creating a new folder for each ticker.

The resulting folder structure is going to look like this:

![folder-structure](https://i.imgur.com/4E8lCTY.png)

We start with creating batches of tickers, with each batch holding 100 tickers. For example, `[A,B,C,D,E,F]` is converted into `[[A,B,C], [D,E,F]]` with a maximum batch length of 3.

In [8]:
# create batches of tickers: [[A,B,C], [D,E,F], ...]
# a single batch has a maximum of max_length_of_batch tickers
def create_batches(tickers = [], max_length_of_batch = 100):
  batches = [[]]

  for ticker in tickers:
    if len(batches[len(batches)-1]) == max_length_of_batch:
      batches.append([])

    batches[len(batches)-1].append(ticker)

  return batches


batches = create_batches(list(holdings['Ticker']))

Next, we define the function `download_10K_metadata(tickers, start_year, end_year)` to download the URLs of the 10-K filings filed by all companies listed in `tickers` in the date range `start_year` and `end_year`.

The complete list of URLs is saved to the file `metadata.csv` on our local disk. Once completed, we can safely re-run `download_10K_metadata()` without having to download all URLs again.

In [9]:
from sec_api import QueryApi, RenderApi
from pathlib import Path
import multiprocessing

api_key='d567bf389458ee8ab52fd3b4ed983d97643aa97d39748ba1aec53156d592bb8f'

queryApi = QueryApi(api_key=api_key)

def download_10K_metadata(tickers = [], start_year = 2020, end_year = 2023):
  if Path('metadata.csv').is_file():
    result = pd.read_csv('metadata.csv')
    return result

  print('✅ Starting download process')

  # create ticker batches, with 100 tickers per batch
  batches = create_batches(tickers)
  frames = []

  for year in range(start_year, end_year + 1):
    for batch in batches:
      tickers_joined = ', '.join(batch)
      ticker_query = 'ticker:({})'.format(tickers_joined)

      query_string = '{ticker_query} AND filedAt:[{start_year}-01-01 TO {end_year}-12-31] AND formType:"10-K" AND NOT formType:"10-K/A" AND NOT formType:NT'.format(ticker_query=ticker_query, start_year=year, end_year=year)

      query = {
        "query": { "query_string": {
            "query": query_string,
            "time_zone": "America/New_York"
        } },
        "from": "0",
        "size": "200",
        "sort": [{ "filedAt": { "order": "desc" } }]
      }

      response = queryApi.get_filings(query)

      filings = response['filings']

      metadata = list(map(lambda f: {'ticker': f['ticker'],
                                     'cik': f['cik'],
                                     'formType': f['formType'],
                                     'filedAt': f['filedAt'],
                                     'filingUrl': f['linkToFilingDetails']}, filings))

      df = pd.DataFrame.from_records(metadata)

      frames.append(df)

    print('✅ Downloaded metadata for year', year)


  result = pd.concat(frames)
  result.to_csv('metadata.csv', index=False)

  number_metadata_downloaded = len(result)
  print('✅ Downloaded completed. Metadata downloaded for {} filings.'.format(number_metadata_downloaded))

  return result


tickers = list(holdings['Ticker'])

metadata = download_10K_metadata(tickers=tickers, start_year=2021, end_year=2023)

✅ Starting download process
✅ Downloaded metadata for year 2021
✅ Downloaded metadata for year 2022
✅ Downloaded metadata for year 2023
✅ Downloaded completed. Metadata downloaded for 7279 filings.


In [10]:
metadata.head(10)

,ticker,cik,formType,filedAt,filingUrl
0,AVGO,1730168,10-K,2021-12-17T16:42:51-05:00,https://www.sec.gov/Archives/edgar/data/173016...
1,AMAT,6951,10-K,2021-12-17T16:14:51-05:00,https://www.sec.gov/Archives/edgar/data/6951/0...
2,DE,315189,10-K,2021-12-16T11:39:34-05:00,https://www.sec.gov/Archives/edgar/data/315189...
3,ADI,6281,10-K,2021-12-03T16:02:52-05:00,https://www.sec.gov/Archives/edgar/data/6281/0...
4,DIS,1744489,10-K,2021-11-24T16:34:25-05:00,https://www.sec.gov/Archives/edgar/data/174448...
5,SBUX,829224,10-K,2021-11-19T16:46:02-05:00,https://www.sec.gov/Archives/edgar/data/829224...
6,V,1403161,10-K,2021-11-18T16:06:50-05:00,https://www.sec.gov/Archives/edgar/data/140316...
7,QCOM,804328,10-K,2021-11-03T16:06:41-04:00,https://www.sec.gov/Archives/edgar/data/804328...
8,AAPL,320193,10-K,2021-10-28T18:04:28-04:00,https://www.sec.gov/Archives/edgar/data/320193...
9,ACN,1467373,10-K,2021-10-15T06:54:29-04:00,https://www.sec.gov/Archives/edgar/data/146737...


In [11]:
print('SEC form types:', list(metadata.formType.unique()))
print('Number of filings:', len(metadata))

SEC form types: ['10-K']
Number of filings: 7279


---

Next, we define the function `download_filing(metadata)`. The function creates a new folder for a ticker, removes the iXBRL inline reader parameter from the filing URL with `.replace('ix?doc=/', '')` and downloads the 10-K filing into the newly created folder.

In [12]:
import os

renderApi = RenderApi(api_key=api_key)

def download_filing(metadata):
  try:
    ticker = metadata['ticker']
    new_folder = "./filings/" + ticker

    if not os.path.isdir(new_folder):
      os.makedirs(new_folder)

    url = metadata['filingUrl'].replace('ix?doc=/', '')
    file_content = renderApi.get_filing(url)
    file_name = url.split("/")[-1]

    with open(new_folder + "/" + file_name, "w") as f:
      f.write(file_content)
  except:
     print('❌ {ticker}: downloaded failed: {url}'.format(ticker=ticker, url=url))

In [34]:
from tqdm import tqdm

for i in tqdm(range(len(metadata)),desc='Processing'):
    download_filing(metadata.iloc[i])

Processing:  43%|████▎     | 3129/7279 [12:33:39<13270:56:06, 11512.14s/it]

❌ PATH: downloaded failed: https://www.sec.gov/Archives/edgar/data/1734722/000173472222000006/path-20220131.htm


Processing:  44%|████▎     | 3178/7279 [12:35:40<2:16:54,  2.00s/it]       

In [13]:
# downloaded sample filing
download_filing(metadata.iloc[0])
print('✅ Sample 10-K filing downloaded for {}'.format(metadata.iloc[0]['ticker']))

✅ Sample 10-K filing downloaded for AVGO


---

We parallelize the download process in order to use the full bandwidth of the Filings Download API. If we were to use a simple `for` loop to iterate over each filing metadata, we would end up downloading all filings one after another, having to wait around 1 hour for the process to complete.

Time calculation (downloading 1 filing at a time):
- Time it takes to download a single filing on average: 550 ms
- Number of filings to download: 6,600
- Total time = `(6600 * 550ms) / 1000ms / 60sec` = 60 min

Instead, we use [`pandarallel`](https://nalepae.github.io/pandarallel/user_guide/), a library used to apply a function in parallel on all rows or columns in a pandas DataFrame. Launching 4 workers allows us to cut the time from 1 hour down to 15 minutes. A worker downloads one filing while all four workers run in parallel.

Time calculation (downloading 4 filings in parallel):
- Time it takes to download a single filing on average: 550 ms
- Number of filings to download: 6,600
- Total time = `(6600 * 550ms) / 1000ms / 60sec / 4 workers` = 15 min

The figure below illustrates the difference between downloading 1 filing at a time vs downloading 4 filings in parallel.

![parallelize-python](https://i.imgur.com/b3vAc6ll.png)

In [14]:
!pip install -q pandarallel

In [16]:
!pip install ipywidgets

   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/139.4 kB ? eta -:--:--
   ----------------------------------- ---- 122.9/139.4 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 139.4/139.4 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   ---------------------------------------- 215.0/215.0 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ----------- ---------------------------- 0.7/2.3 MB 13.7 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.3 MB 16.0 MB/s eta 0:00:01
   ---------------------------------- ----- 2.0/2.3 MB 14.0 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 14.7 MB/s eta 0:00:00


In [18]:
import os

In [21]:
metadata.head(50)

,ticker,cik,formType,filedAt,filingUrl
0,AVGO,1730168,10-K,2021-12-17T16:42:51-05:00,https://www.sec.gov/Archives/edgar/data/173016...
1,AMAT,6951,10-K,2021-12-17T16:14:51-05:00,https://www.sec.gov/Archives/edgar/data/6951/0...
2,DE,315189,10-K,2021-12-16T11:39:34-05:00,https://www.sec.gov/Archives/edgar/data/315189...
3,ADI,6281,10-K,2021-12-03T16:02:52-05:00,https://www.sec.gov/Archives/edgar/data/6281/0...
4,DIS,1744489,10-K,2021-11-24T16:34:25-05:00,https://www.sec.gov/Archives/edgar/data/174448...
5,SBUX,829224,10-K,2021-11-19T16:46:02-05:00,https://www.sec.gov/Archives/edgar/data/829224...
6,V,1403161,10-K,2021-11-18T16:06:50-05:00,https://www.sec.gov/Archives/edgar/data/140316...
7,QCOM,804328,10-K,2021-11-03T16:06:41-04:00,https://www.sec.gov/Archives/edgar/data/804328...
8,AAPL,320193,10-K,2021-10-28T18:04:28-04:00,https://www.sec.gov/Archives/edgar/data/320193...
9,ACN,1467373,10-K,2021-10-15T06:54:29-04:00,https://www.sec.gov/Archives/edgar/data/146737...


In [27]:
from pandarallel import pandarallel

number_of_workers = 4
pandarallel.initialize(progress_bar=True, nb_workers=number_of_workers, verbose=0)

# uncomment to run a quick sample and download 50 filings
sample = metadata.head(50)
sample.parallel_apply(download_filing, axis=1)

# download all filings
#metadata.parallel_apply(download_filing, axis=1)

print('✅ Download completed')

✅ Download completed
